In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os, gc
import plotly.express as px
import plotly.graph_objects as go
from lightgbm import LGBMClassifier, early_stopping, log_evaluation, plot_importance
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc

# **Read File**

In [3]:
TRAIN_PATH = '../input/pa-amex-default-dataset/train.parquet'

In [4]:
%%time
def read_file(path = '', usecols = None):
    if usecols is not None: df = pd.read_parquet(path, columns=usecols)
    else: df = pd.read_parquet(path)
    
    print('shape of data:', df.shape)
    print("The training data begins on {} and ends on {}.".format(df['S_2'].min(),df['S_2'].max()))

    df = df.sort_values(['customer_ID','S_2'])
    df_out = df.groupby(['customer_ID']).nth(-1).reset_index(drop=True)
    print("There are {:,.0f} customers in the training set and {} features.".format(df_out.shape[0],df_out.shape[1]))
    del df
    _ = gc.collect()
    return df_out

print('Reading train data...')
train = read_file(path = TRAIN_PATH)

In [5]:
train.info(max_cols = 200, show_counts = True)

In [6]:
train.head()

# **Evaluation Metric**

In [7]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

# **Training Model**

In [8]:
# Create the arrays for features and the target: X, y
features = [x for x in train.columns.values if x not in ['customer_ID', 'target', 'S_2']]
X, y = train[features], train['target']

# Create the training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.2, 
                                                    random_state=100,
                                                    stratify=y)

In [9]:
clf = LGBMClassifier(
    n_estimators=50000,
    device='gpu',
    random_state=69420,
    extra_trees=True
)

In [10]:
%%time
clf.fit(
    X_train, y_train, 
    eval_set=[(X_test,y_test)],
    callbacks=[early_stopping(50), log_evaluation(0)]
)

In [11]:
from lightgbm import plot_importance
# Review the important features
def plot_features(booster, figsize, max_num_features=20):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax, max_num_features=max_num_features)
plot_features(clf, (10,14))
plt.show()

In [12]:
y_pred = pd.DataFrame(y_test.copy(deep=True))
y_pred = y_pred.rename(columns={'target':'prediction'})
y_pred

In [13]:
%%time
y_pred["prediction"] = clf.predict_proba(X_test)[:,1]
y_pred

In [14]:
%%time
y_test = pd.DataFrame(y_test)
amex_metric(y_test, y_pred)